<a href="https://colab.research.google.com/github/ZAIN007777/Skin-Disease-Dectection-App/blob/main/skincare_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
import time

# Load and prepare data
df = pd.read_csv("dataset.csv")
questions = df["question"].dropna().tolist()
answers = df["answer"].dropna().tolist()

# Split into train and test sets
train_questions, test_questions, train_answers, test_answers = train_test_split(
    questions, answers, test_size=0.2, random_state=42
)

# Load model and embed training questions
embedder = SentenceTransformer("all-mpnet-base-v2")
train_embeddings = embedder.encode(train_questions, convert_to_numpy=True)
train_embeddings = train_embeddings / np.linalg.norm(train_embeddings, axis=1, keepdims=True)

# Create FAISS index
index = faiss.IndexFlatL2(768)
index.add(train_embeddings)

# Chatbot response function with evaluation
def get_answer_with_evaluation(question, true_answer=None):
    start_time = time.time()

    # Encode input
    question_embedding = embedder.encode([question], convert_to_numpy=True)
    question_embedding = question_embedding / np.linalg.norm(question_embedding, axis=1, keepdims=True)

    # FAISS search for top-1
    distances, indices = index.search(question_embedding, k=1)
    retrieval_time = time.time() - start_time

    # Get top answer
    top_answer = train_answers[indices[0][0]]

    # Compute confidence
    confidence = 1 / (1 + distances[0][0])

    # Semantic consistency (similarity of answer with itself = 1.0)
    consistency_score = 1.0

    # Coverage: similarity between question and top answer
    top_answer_embedding = embedder.encode([top_answer], convert_to_numpy=True)
    coverage_score = util.cos_sim(question_embedding, top_answer_embedding).item()

    return {
        'answer': top_answer,
        'evaluation': {
            'retrieval_time': retrieval_time,
            'confidence': float(confidence),
            'semantic_similarity': float(consistency_score),
            'coverage': float(coverage_score)
        }
    }

# Chat loop
if __name__ == "__main__":
    print("\n=== SkinCare Chatbot (Top-1 Answer Only) ===")
    while True:
        user_input = input("\nAsk a skincare question (or type 'exit'): ")
        if user_input.strip().lower() == 'exit':
            break

        # Optionally match to test set
        user_embedding = embedder.encode([user_input], convert_to_numpy=True)
        test_embeddings = embedder.encode(test_questions, convert_to_numpy=True)
        user_embedding_norm = user_embedding / np.linalg.norm(user_embedding)
        test_embeddings_norm = test_embeddings / np.linalg.norm(test_embeddings, axis=1, keepdims=True)
        cos_similarities = np.dot(test_embeddings_norm, user_embedding_norm.T).squeeze()

        top_match_idx = np.argmax(cos_similarities)
        max_sim = cos_similarities[top_match_idx]
        true_answer = test_answers[top_match_idx] if max_sim > 0.7 else None

        # Get response
        response = get_answer_with_evaluation(user_input, true_answer=true_answer)

        # Output
        print(f"\nAnswer: {response['answer']}")
        print("\n--- Evaluation ---")
        print(f"Retrieval Time: {response['evaluation']['retrieval_time']:.4f}s")
        print(f"Confidence: {response['evaluation']['confidence']:.2%}")
        print(f"Semantic Similarity (Consistency): {response['evaluation']['semantic_similarity']:.4f}")



=== SkinCare Chatbot (Top-1 Answer Only) ===

Ask a skincare question (or type 'exit'): what is acne?

Answer: Acne is a skin condition that occurs when hair follicles become plugged with oil and dead skin cells. It commonly causes whiteheads, blackheads, or pimples, and usually appears on the face, forehead, chest, upper back, and shoulders.

Acne is most common among teenagers, but it affects people of all ages. Factors that can trigger or worsen acne include hormones, certain medications, diet, stress, and poor hygiene.

Acne treatment depends on its severity and persistence. Mild acne can be treated with over-the-counter products containing benzoyl peroxide or salicylic acid. More severe cases might require topical or oral prescription medications such as antibiotics, retinoids or vitamin A derivatives, azelaic acid, or dapsone. In some cases, hormonal therapy might be beneficial. It's always a good idea to consult a dermatologist for an effective treatment plan

--- Evaluation --